In [10]:
print('hello')

hello


In [11]:
!pip install python-dotenv

import os
from dotenv import load_dotenv
import requests
import pandas as pd

In [12]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

In [13]:
url = "https://gateway.thegraph.com/api/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

headers = {
    "Authorization": f"Bearer {API_KEY}"
}

# fetch top 1000 pools based on Volume (in USD) in Desc. order 
query = """
{
  pools(first: 1000, orderBy: volumeUSD, orderDirection: desc) {
    id
    token0 { symbol }
    token1 { symbol }
    feeTier
    volumeUSD
    totalValueLockedUSD
    token0Price
    token1Price
  }
}
"""

In [14]:

response = requests.post(url, json={"query": query}, headers=headers)
print(response.json())

{'data': {'pools': [{'feeTier': '500', 'id': '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640', 'token0': {'symbol': 'USDC'}, 'token0Price': '1470.805081462323888928571819680618', 'token1': {'symbol': 'WETH'}, 'token1Price': '0.0006798997451149450480834406737692959', 'totalValueLockedUSD': '331357581.5148106492468435956170674', 'volumeUSD': '550489571350.3370197487228449723638'}, {'feeTier': '500', 'id': '0x11b815efb8f581194ae79006d24e0d814b7697f6', 'token0': {'symbol': 'WETH'}, 'token0Price': '0.0006794570935512450048818991960354633', 'token1': {'symbol': 'USDT'}, 'token1Price': '1471.763279081255899168092315377131', 'totalValueLockedUSD': '60849841.23850175829391081927845663', 'volumeUSD': '119578316184.2202369744374338270036'}, {'feeTier': '100', 'id': '0x3416cf6c708da44db2624d63ea0aaef7113527c6', 'token0': {'symbol': 'USDC'}, 'token0Price': '0.9990577310599268020645307198879425', 'token1': {'symbol': 'USDT'}, 'token1Price': '1.000943157648230718224398695503111', 'totalValueLockedUSD': '

In [15]:
response = requests.post(url, json={"query": query}, headers=headers)
data = response.json()

# Extract pool data
pools = data["data"]["pools"]

# Normalize nested token symbols into flat fields
clean_data = []
for pool in pools:
    clean_data.append({
        "id": pool["id"],
        "token0": pool["token0"]["symbol"],
        "token1": pool["token1"]["symbol"],
        "feeTier": pool["feeTier"],
        "volumeUSD": pool["volumeUSD"],
        "totalValueLockedUSD": pool["totalValueLockedUSD"],
        "token0Price": pool["token0Price"],
        "token1Price": pool["token1Price"]
    })

In [16]:
# Load into DataFrame
df = pd.DataFrame(clean_data)

# Convert numeric columns
numeric_fields = ["volumeUSD", "totalValueLockedUSD", "token0Price", "token1Price", "feeTier"]
df[numeric_fields] = df[numeric_fields].astype(float)

# Optional: sort by volume
df = df.sort_values(by="volumeUSD", ascending=False)

# Show the result
df.head(5)

,id,token0,token1,feeTier,volumeUSD,totalValueLockedUSD,token0Price,token1Price
0,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,WETH,500.0,5.504865e+11,3.313767e+08,1470.805081,0.000680
1,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,USDT,500.0,1.195783e+11,6.085535e+07,0.000679,1471.763279
2,0x3416cf6c708da44db2624d63ea0aaef7113527c6,USDC,USDT,100.0,9.793214e+10,4.491304e+07,0.999058,1.000943
3,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,WBTC,WETH,500.0,8.677259e+10,7.920803e+07,0.019131,52.270361
4,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,USDC,WETH,3000.0,8.308687e+10,2.360727e+08,1474.736487,0.000678


In [17]:
df.shape

(1000, 8)

In [18]:
import pandas as pd

# Combine token0 and token1 into a single Series
all_tokens = pd.concat([df['token0'], df['token1']])

# Count appearances of each token
token_counts = all_tokens.value_counts()

# Show top tokens
print(token_counts.head(10))

WETH    727
USDC    159
USDT     75
WBTC     38
DAI      23
FEI       9
UST       8
UNI       8
FRAX      6
FUN       6
Name: count, dtype: int64
